In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import sklearn
import skimage
import scipy.stats
import pywt
import os

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", message='.*greycoprops.*') 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Applied AI/train.zip" -d "/content/train/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/train/P/P03803_1.jpeg  
  inflating: /content/train/P/P03803_2.jpeg  
  inflating: /content/train/P/P03804_1.jpeg  
  inflating: /content/train/P/P03809_1.jpeg  
  inflating: /content/train/P/P03809_2.jpeg  
  inflating: /content/train/P/P03811_1.jpeg  
  inflating: /content/train/P/P03811_2.jpeg  
  inflating: /content/train/P/P03813_1.jpeg  
  inflating: /content/train/P/P03814_1.jpeg  
  inflating: /content/train/P/P03814_2.jpeg  
  inflating: /content/train/P/P03815_1.jpeg  
  inflating: /content/train/P/P03817_1.jpeg  
  inflating: /content/train/P/P03817_2.jpeg  
  inflating: /content/train/P/P03821_1.jpeg  
  inflating: /content/train/P/P03821_2.jpeg  
  inflating: /content/train/P/P03828_1.jpeg  
  inflating: /content/train/P/P03828_2.jpeg  
  inflating: /content/train/P/P03830_1.jpeg  
  inflating: /content/train/P/P03831_1.jpeg  
  inflating: /content/train/P/P03838_1.jpeg  
  inflating: /content/train/P

In [ ]:
dataframe = pd.read_csv('/content/drive/My Drive/Applied AI/quality_annotated_dataframe.csv')

In [ ]:
def preprocessing(image):
  if image.shape[-1] == 3:
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  
  image = np.uint8(image)

  # Check if background is Standard Radiological Format compliant, otherwise invert colors
  # (the check is performed on the median color along the spine)
  buffer = image / np.max(image)

  median_spine = np.median(buffer[:, buffer.shape[1] // 2])

  if median_spine < 0.5:
    image = cv2.bitwise_not(image)

  # Mist reduction and gamma transformation
  img_equalized = cv2.equalizeHist(image)
  gamma = 1.5
  inv_gamma = 1.0 / gamma
  table = np.array([((i / 255.0) ** inv_gamma) * 255
                    for i in np.arange(0, 256)]).astype('uint8')
  img_gamma = cv2.LUT(img_equalized, table)

  # Apply Adaptive Contrast Equalization
  clahe = cv2.createCLAHE(clipLimit=40.0, tileGridSize=(8,8))
  clahe_img = clahe.apply(img_gamma)

  # Apply Gaussian Smoothing
  gaussian_img = cv2.GaussianBlur(clahe_img, (5,5), 0)

  return gaussian_img

In [ ]:
dataframe_quality = dataframe[dataframe['quality'] == 0]

filenames = dataframe_quality['file'].to_numpy()
to_remove = np.zeros(filenames.shape[0])

num_duplicates = 0

# Let's remove duplicates
for index in range(filenames.shape[0]):
    # Ignore the last element
    if index == filenames.shape[0] - 1:
        break
    
    filename = filenames[index]
    next_filename = filenames[index + 1]
    
    # Remove extension
    filename = filename.replace('.png', '')
    filename = filename.replace('.jpeg', '')
    next_filename = next_filename.replace('.png', '')
    next_filename = next_filename.replace('.jpeg', '')
    
    # If consecutive filenames refer to the same patient
    if filename[:-2] == next_filename[:-2]:
        to_remove[index + 1] = 1
        num_duplicates = num_duplicates + 1
        index = index + 1
        
dataframe_quality = dataframe_quality[to_remove == 0]
        
print('Removed %d duplicates' % num_duplicates)

dataframe_path = dataframe_quality.copy()
dataframe_path['path'] = '/content/train/' + dataframe_path['label'] + '/' + dataframe_path['file']

Removed 1293 duplicates


In [ ]:
total_samples = dataframe_path.shape[0]

features = np.empty(shape=(total_samples, 68)) # Features
labels = np.zeros(shape=(total_samples, 3)) # One-hot-encoding

In [ ]:
ordinal_dict = {'N':0,
                'P':1,
                'T':2}

def func_dict(x):
  return ordinal_dict[x]

In [ ]:
paths = dataframe_path['path'].to_numpy()
ordinal_labels = dataframe_path['label'].apply(func_dict).to_numpy().astype(np.int32)

In [ ]:
test_image = cv2.imread(paths[0], cv2.IMREAD_GRAYSCALE)
test_processed = preprocessing(test_image)

# Let's print the list of features being computed
features_names = ['mean intensity',
                  'std intensity',
                  'median intesity',
                  'q25 intensity', 'q50 intensity', 'q75 intensity']

## Local Binary Patterns
# Parameters
num_points = 12
radius = 5

lbp = skimage.feature.local_binary_pattern(test_processed, num_points, radius, method="uniform")
(hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, num_points + 3), range=(0, num_points + 2))

num_hist_features = hist.size

for i in range(num_hist_features):
  features_names.append(f'lbp_hist_{i}')

# GCLM features
for distance in ['1', '2']:
  for angle in ['0', '90']:
    for feature in ['contrast', 'dissimilarity', 'homogeneity', 'asm', 'energy', 'correlation']:
      features_names.append(f'glcm_d{distance}_a{angle}_{feature}')

# Wavelet-based texture features
for matrix in ['cA', 'cH', 'cV', 'cD']:
  for feature in ['energy', 'entropy', 'mean', 'variance', 'skewness', 'kurtosis']:
    features_names.append(f'wavelet_{matrix}_{feature}')

In [ ]:
for k, feature_name in enumerate(features_names):
  print(f'Feature {k} is {feature_name}')

Feature 0 is mean intensity
Feature 1 is std intensity
Feature 2 is median intesity
Feature 3 is q25 intensity
Feature 4 is q50 intensity
Feature 5 is q75 intensity
Feature 6 is lbp_hist_0
Feature 7 is lbp_hist_1
Feature 8 is lbp_hist_2
Feature 9 is lbp_hist_3
Feature 10 is lbp_hist_4
Feature 11 is lbp_hist_5
Feature 12 is lbp_hist_6
Feature 13 is lbp_hist_7
Feature 14 is lbp_hist_8
Feature 15 is lbp_hist_9
Feature 16 is lbp_hist_10
Feature 17 is lbp_hist_11
Feature 18 is lbp_hist_12
Feature 19 is lbp_hist_13
Feature 20 is glcm_d1_a0_contrast
Feature 21 is glcm_d1_a0_dissimilarity
Feature 22 is glcm_d1_a0_homogeneity
Feature 23 is glcm_d1_a0_asm
Feature 24 is glcm_d1_a0_energy
Feature 25 is glcm_d1_a0_correlation
Feature 26 is glcm_d1_a90_contrast
Feature 27 is glcm_d1_a90_dissimilarity
Feature 28 is glcm_d1_a90_homogeneity
Feature 29 is glcm_d1_a90_asm
Feature 30 is glcm_d1_a90_energy
Feature 31 is glcm_d1_a90_correlation
Feature 32 is glcm_d2_a0_contrast
Feature 33 is glcm_d2_a0_diss

In [ ]:
num_examples = paths.shape[0]
num_features = len(features_names)

for index in range(num_examples):

  image = cv2.imread(paths[index], cv2.IMREAD_GRAYSCALE)
  processed = preprocessing(image)

  # Average intensity
  features[index, 0] = np.mean(processed)

  # Standard deviation of intensity
  features[index, 1] = np.std(processed)

  # Median intensity
  features[index, 2] = np.median(processed)

  # Quartiles of the intensities
  # 25th, 50th, and 75th percentiles
  q25, q50, q75 = np.quantile(processed, [0.25, 0.5, 0.75])
  features[index, 3] = q25
  features[index, 4] = q50
  features[index, 5] = q75

  ## Local Binary Patterns
  # Parameters
  num_points = 12
  radius = 5

  lbp = skimage.feature.local_binary_pattern(processed, num_points, radius, method="uniform")
  (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, num_points + 3), range=(0, num_points + 2))

  # Histogram normalization
  hist = hist.astype("float")
  hist /= (hist.sum() + 1e-7) # 1e-7 is an epsilon to avoid division by zero

  features[index, 6:20] = hist

  # Compute the GLCM (Grey Level Co-occurrence Matrix) to extract properties
  glcm = skimage.feature.graycomatrix(processed, distances=[1, 2], angles=[0, np.pi/2], symmetric=True, normed=True)

  contrast = skimage.feature.graycoprops(glcm, 'contrast').flatten()
  dissimilarity = skimage.feature.graycoprops(glcm, 'dissimilarity').flatten()
  homogeneity = skimage.feature.graycoprops(glcm, 'homogeneity').flatten()
  asm = skimage.feature.graycoprops(glcm, 'ASM').flatten()
  energy = skimage.feature.graycoprops(glcm, 'energy').flatten()
  correlation = skimage.feature.graycoprops(glcm, 'correlation').flatten()

  to_add = np.concatenate([contrast, dissimilarity, homogeneity, asm, energy, correlation])

  features[index, 20:44] = to_add

  ### Histogram of Oriented Gradients is also an option, but it's really large, so let's avoid it for now

  ## Wavelet-based features

  # Compute the 2D discrete wavelet transform of the image using the 'db1' wavelet
  coeffs = pywt.dwt2(processed, 'db1')

  # Extract the approximation (low-pass) coefficients and the detail (high-pass) coefficients
  cA, (cH, cV, cD) = coeffs

  # Compute the wavelet-based texture features
  matrices = [cA, cH, cV, cD]

  wavelet_features = np.empty(shape=(4, 6))

  for k, matrix in enumerate(matrices):
    sqr_matrix = matrix ** 2

    energy = np.sum(sqr_matrix)

    # Estimate the histogram of the data
    hist, bin_edges = np.histogram(matrix, bins=32, density=True)
    bin_values = np.array([(bin_edges[i] + bin_edges[i+1]) / 2 for i in range(hist.shape[0])])

    hist = hist + 1e-7 # Avoid division by zero
    bin_values = bin_values + 1e-7 # Avoid division by zero

    entropy = scipy.stats.entropy(hist, bin_values)
    
    mean = np.mean(matrix)
    variance = np.var(matrix)
    skewness = scipy.stats.skew(matrix, axis=None)
    kurtosis = scipy.stats.kurtosis(matrix, axis=None)
    
    # Add features to the matrix of features
    wavelet_features[k, 0] = energy
    wavelet_features[k, 1] = entropy
    wavelet_features[k, 2] = mean
    wavelet_features[k, 3] = variance
    wavelet_features[k, 4] = skewness
    wavelet_features[k, 5] = kurtosis

  features[index, 44:68] = wavelet_features.flatten()

  sample_label = ordinal_labels[index]
  ordinal_label = sample_label

  # Set label in one-hot encoding
  labels[index, ordinal_label] = 1

  print('Finished image {} : {}/{}'.format(paths[index], index, total_samples))

Streaming output truncated to the last 5000 lines.
Finished image /content/train/P/P08531_2.jpeg : 5792/10792
Finished image /content/train/T/P08533_1.png : 5793/10792
Finished image /content/train/P/P08534_1.jpeg : 5794/10792
Finished image /content/train/T/P08535_1.png : 5795/10792
Finished image /content/train/N/P08537_1.png : 5796/10792
Finished image /content/train/N/P08539_1.png : 5797/10792
Finished image /content/train/N/P08540_1.png : 5798/10792
Finished image /content/train/N/P08542_1.png : 5799/10792
Finished image /content/train/N/P08543_1.png : 5800/10792
Finished image /content/train/N/P08544_1.png : 5801/10792
Finished image /content/train/N/P08545_2.png : 5802/10792
Finished image /content/train/P/P08548_1.jpeg : 5803/10792
Finished image /content/train/T/P08549_1.png : 5804/10792
Finished image /content/train/P/P08550_2.jpeg : 5805/10792
Finished image /content/train/N/P08551_1.png : 5806/10792
Finished image /content/train/P/P08552_1.jpeg : 5807/10792
Finished image /

In [ ]:
handcrafted_features_df = pd.DataFrame(data=features, columns=features_names)

In [ ]:
handcrafted_features_df.to_csv('/content/drive/My Drive/Applied AI/handcrafted_features.csv', sep=',', index=False)
np.save("/content/drive/MyDrive/Applied AI/handcrafted_labels.npy", labels)

In [ ]:
handcrafted_features_df

,mean intensity,std intensity,median intesity,q25 intensity,q50 intensity,q75 intensity,lbp_hist_0,lbp_hist_1,lbp_hist_2,lbp_hist_3,...,wavelet_cV_mean,wavelet_cV_variance,wavelet_cV_skewness,wavelet_cV_kurtosis,wavelet_cD_energy,wavelet_cD_entropy,wavelet_cD_mean,wavelet_cD_variance,wavelet_cD_skewness,wavelet_cD_kurtosis
0,122.658481,61.189018,124.0,73.0,124.0,167.0,0.026569,0.035406,0.035962,0.035587,...,-0.023713,67.859981,0.004813,3.005386,71017.75,inf,0.000462,1.775444,0.001266,4.700534
1,122.726644,60.322904,122.0,76.0,122.0,166.0,0.024725,0.036837,0.029544,0.031519,...,0.095963,59.076197,0.128725,4.111251,60884.25,inf,-0.000588,1.522106,0.107907,7.923528
2,130.004506,58.470367,127.0,88.0,127.0,172.0,0.024437,0.042050,0.035594,0.034137,...,0.409513,70.416831,0.567822,5.006772,47047.25,inf,-0.002237,1.176176,0.065474,5.781026
3,116.356489,62.618169,111.0,62.0,111.0,161.0,0.004394,0.005813,0.009631,0.017216,...,0.002734,5.468566,0.009405,10.538703,42703.75,inf,0.000005,0.090744,-0.047113,2.710796
4,130.273293,60.258907,129.0,88.0,129.0,173.0,0.002923,0.004890,0.007945,0.012362,...,-0.006963,2.413165,0.063858,7.304261,161221.00,inf,-0.000504,0.094233,-0.010963,2.200583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10787,136.472941,63.905027,137.0,90.0,137.0,182.0,0.029175,0.028419,0.034163,0.038325,...,0.056653,29.750154,0.145457,12.621610,120727.00,inf,-0.000349,0.467818,-0.030033,13.270150
10788,130.363804,61.248853,128.0,86.0,128.0,175.0,0.002364,0.003933,0.006901,0.011253,...,-0.004658,2.615156,0.315068,25.220991,170397.50,inf,0.000010,0.091052,-0.000460,2.693692
10789,126.151538,63.050819,125.0,77.0,125.0,172.0,0.003820,0.005023,0.008664,0.015362,...,0.001419,4.476452,0.024346,4.324561,60136.00,inf,0.000179,0.091431,0.000049,1.160160
10790,131.124891,61.843324,131.0,88.0,131.0,177.0,0.026491,0.019120,0.032226,0.043438,...,-0.002256,13.041549,-0.081068,10.843171,551471.25,inf,-0.000011,0.246411,0.001194,4.532587
